# In this notebook we'll explore the results of stackoverflow surveys from 2019, 2020, and 2021

## The questions we want to answer are:

## 1. Are the programming languages students are yearning to learn the same 
##  as those who are already developers and have a good salary over time?

## 2. Do programmers who are compensated in USD earn higher salaries than those who earn in BRL?
### 2.1 Are Brazilians' salaries from those who earn in USD higher than those who are compensated in BRL?

## 3. Are salaries growing faster in Brazil or in the US?

In [1]:
from statistics import mean, median

import plotly.graph_objects as go
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# Reading data
df_schema = pd.read_csv('survey_results_schema.csv')
df_schema.name = 'schema'
df_answers_2019 = pd.read_csv('survey_results_public_2019.csv')
df_answers_2019.name = '2019'
df_answers_2020 = pd.read_csv('survey_results_public_2020.csv')
df_answers_2020.name = '2020'
df_answers = pd.read_csv('survey_results_public_2021.csv')
df_answers.name = '2021'
df_answers.head()

,ResponseId,MainBranch,Employment,Country,US_State,UK_Country,EdLevel,Age1stCode,LearnCode,YearsCode,YearsCodePro,DevType,OrgSize,Currency,CompTotal,CompFreq,LanguageHaveWorkedWith,LanguageWantToWorkWith,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,PlatformHaveWorkedWith,PlatformWantToWorkWith,WebframeHaveWorkedWith,WebframeWantToWorkWith,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,OpSys,NEWStuck,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOComm,NEWOtherComms,Age,Gender,Trans,Sexuality,Ethnicity,Accessibility,MentalHealth,SurveyLength,SurveyEase,ConvertedCompYearly
0,1,I am a developer by profession,"Independent contractor, freelancer, or self-em...",Slovakia,NaN,NaN,"Secondary school (e.g. American high school, G...",18 - 24 years,Coding Bootcamp;Other online resources (ex: vi...,NaN,NaN,"Developer, mobile",20 to 99 employees,EUR European Euro,4800.00,Monthly,C++;HTML/CSS;JavaScript;Objective-C;PHP;Swift,Swift,PostgreSQL;SQLite,SQLite,NaN,NaN,Laravel;Symfony,NaN,NaN,NaN,NaN,NaN,PHPStorm;Xcode,Atom;Xcode,MacOS,Call a coworker or friend;Visit Stack Overflow...,Stack Overflow,Multiple times per day,Yes,A few times per month or weekly,"Yes, definitely",No,25-34 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,62268.00
1,2,I am a student who is learning to code,"Student, full-time",Netherlands,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",7,NaN,NaN,NaN,NaN,NaN,NaN,JavaScript;Python,NaN,PostgreSQL,NaN,NaN,NaN,Angular;Flask;Vue.js,NaN,Cordova,NaN,Docker;Git;Yarn,Git,Android Studio;IntelliJ;Notepad++;PyCharm,NaN,Windows,Visit Stack Overflow;Google it,Stack Overflow,Daily or almost daily,Yes,Daily or almost daily,"Yes, definitely",No,18-24 years old,Man,No,Straight / Heterosexual,White or of European descent,None of the above,None of the above,Appropriate in length,Easy,NaN
2,3,"I am not primarily a developer, but I write co...","Student, full-time",Russian Federation,NaN,NaN,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",11 - 17 years,"Other online resources (ex: videos, blogs, etc...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assembly;C;Python;R;Rust,Julia;Python;Rust,SQLite,SQLite,Heroku,NaN,Flask,Flask,NumPy;Pandas;TensorFlow;Torch/PyTorch,Keras;NumPy;Pandas;TensorFlow;Torch/PyTorch,NaN,NaN,IPython/Jupyter;PyCharm;RStudio;Sublime Text;V...,IPython/Jupyter;RStudio;Sublime Text;Visual St...,MacOS,Visit Stack Overflow;Google it;Watch help / tu...,Stack Overflow;Stack Exchange,Multiple times per day,Yes,Multiple times per day,"Yes, definitely",Yes,18-24 years old,Man,No,Prefer not to say,Prefer not to say,None of the above,None of the above,Appropriate in length,Easy,NaN
3,4,I am a developer by profession,Employed full-time,Austria,NaN,NaN,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",11 - 17 years,NaN,NaN,NaN,"Developer, front-end",100 to 499 employees,EUR European Euro,NaN,Monthly,JavaScript;TypeScript,JavaScript;TypeScript,NaN,NaN,NaN,NaN,Angular;jQuery,Angular;jQuery,NaN,NaN,NaN,NaN,NaN,NaN,Windows,Call a coworker or friend;Visit Stack Overflow...,Stack Overflow,Daily or almost daily,Yes,Daily or almost daily,Neutral,No,35-44 years old,Man,No,Straight / Heterosexual,White or of European descent,I am deaf / hard of hearing,NaN,Appropriate in length,Neither easy nor difficult,NaN
4,5,I am a developer by profession,"Independent contractor, freelancer, or self-em...",United Kingdom of Great Britain and Northern I...,NaN,England,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)",5 - 10 years,Friend or family member,17,10,"Developer, desktop or enterprise applications;...","Just me - I am a freelancer, sole proprietor, ...",GBP\tPound sterling,NaN,NaN,Bash/Shell;HTML/CSS;Python;SQL,Bash/Shell;HTML/CSS;Python;SQL,Elasticsearch;PostgreSQL;Redis,Cassandra;Elasticsearch;PostgreSQL;Redis,

# 1. Are the programming languages students are yearning to learn the same 
#    as those who are already developers and have a good salary?

## Functions


In [3]:
def get_cols_names(df):
    '''
    INPUT 
        df - a dataframe with answers to the StackOverflow survey
    OUTPUT
        want_to_learn_col, worked_with_col, salary_col, currency_col -
        the names of the columns of interest, which may vary depending 
        on the year of the survey
    '''
    if df.name in ['2019','2020']:
        want_to_learn_col = 'LanguageDesireNextYear'
        worked_with_col = 'LanguageWorkedWith'
        salary_col = 'ConvertedComp'
        currency_col = 'CurrencySymbol'
    elif df.name == '2021':
        want_to_learn_col = 'LanguageWantToWorkWith'
        worked_with_col = 'LanguageHaveWorkedWith'
        salary_col = 'ConvertedCompYearly'
        currency_col = 'Currency'
    return want_to_learn_col, worked_with_col, salary_col, currency_col

def drop_outliers(df,column,trim_pct=.15):
    '''
    INPUT 
        df - a dataframe with answers to the StackOverflow survey
    OUTPUT
        trim_pct - percentage to trim from the top of the input dataframe
        column - the column to sort the value in the dataframe before trimming them
        df - the input dataframe trimmed with top `trim_pct` of rows according to `column`
    '''
    return df.sort_values(column,ascending=False).drop([x for x in range(int(len(df)*trim_pct))])

def create_df_by_main_branch(df,main_branch):
    return df[df['MainBranch'] == main_branch]

def create_wanted_languages_df(df,df_type,want_to_learn_col,worked_with_col,salary_col):
    '''
    INPUT 
        df - a dataframe with answers to the StackOverflow survey
        df_type - if creating a dataframe from students answers to the survey we must not
                require they have already worked with any programming language, which is not the
                case with professional devs
        want_to_learn_col - the column with the programming languages the respondent want to learn/work with
                            in the future
        worked_with_col - the column with the programming languages the respondent already learned/worked with
        salary_col - the column with the salaries from the respondents
    OUTPUT
        df_wanted_languages - a dataframe with how many respondents want to learn some programming languages
                            and their salaries
    '''
    wanted_languages = {}
    
    if df_type == 'students':
        df[worked_with_col] = ''

    for i, row in df.iterrows():
        p_languages = row[want_to_learn_col].split(';')
        for pl in p_languages:
            if pl not in wanted_languages and pl not in row[worked_with_col]:
                wanted_languages[pl] = {'respondents':1, 'salaries':[row[salary_col]]}
                continue
            if pl in wanted_languages and pl not in row[worked_with_col]:
                wanted_languages[pl]['respondents'] += 1
                wanted_languages[pl]['salaries'].append(row[salary_col])

    df_wanted_languages = pd.DataFrame.from_dict(wanted_languages, orient='index').reset_index()\
                            .rename(columns={'index': want_to_learn_col, 0: 'count'})
    return df_wanted_languages

def calculate_salaries_statistics(df,trim_pct=.1):
    '''
    INPUT 
        df - a dataframe with answers to the StackOverflow survey
        trim_pct - percentage to trim from bottom and top of the input dataframe to calculate statistics
    OUTPUT
        df - a dataframe with salaries statistics
    '''
    df['respondents_for_calc'] = df['salaries'].apply(lambda x: len(x[int(len(x)*trim_pct):-int(len(x)*trim_pct)]))
    df = df.loc[df['respondents_for_calc'] >= 1, :]
    df['mean_salary'] = df['salaries'].apply(lambda x: mean(sorted(x)[int(len(x)*trim_pct):-int(len(x)*trim_pct)]))
    df['median_salary'] = df['salaries'].apply(lambda x: median(sorted(x)[int(len(x)*trim_pct):-int(len(x)*trim_pct)]))
    df['median_pct_of_mean'] = round(df['median_salary'] / df['mean_salary'],2)
    df = df.drop('salaries',axis=1)
    return df

def filter_df_languages_devs(df,min_median_mean_ratio=.7,min_resp=100,n_head=10):
    '''
    INPUT 
        df - a dataframe with answers to the StackOverflow survey
        min_median_mean_ratio - the minimum median to mean of salaries ratio, the closer to 1 the value,
                                the more closely distributed the values are
        min_resp - the minimum number of respondents that want to learn a new programming language
        n_head - the top n programming languages based on the number of respondents which want to learn them
    OUTPUT
        df - the input dataframe filtered based on `min_median_mean_ratio`, `min_resp` and `n_head`
    '''
    df = df[(df['median_pct_of_mean'] >= min_median_mean_ratio) & (df['respondents_for_calc'] >= min_resp)]\
            .sort_values('respondents_for_calc',ascending=False).head(n_head)
    return df

In [4]:
dfs = [
    df_answers_2019,
    df_answers_2020,
    df_answers
]

df_devs_all = pd.DataFrame()
df_students_all = pd.DataFrame()

for df in dfs:
    # Getting proper column names for each dataframe
    want_to_learn_col, worked_with_col, salary_col, currency_col = get_cols_names(df)

    # Droping rows with top 15% salaries (maybe they were typed incorrectly or are much higher than typical salaries)
    df_answers_clean = drop_outliers(df,salary_col)

    # Since we want to know what devs with top earnings want to learn next, let's analyze the answers from devs
    # which salaries are amongst the top 20%
    df_answers_devs = create_df_by_main_branch(df_answers_clean,'I am a developer by profession')\
                        .dropna(subset=[worked_with_col,want_to_learn_col])
    df_answers_devs = df_answers_devs.sort_values(salary_col,ascending=False).head(int(len(df_answers_devs)*.2))

    # Creating a dataframe with the languages well compensated devs want to learn
    df_languages_devs = create_wanted_languages_df(df_answers_devs,'devs',want_to_learn_col,worked_with_col,salary_col)
    df_languages_devs = calculate_salaries_statistics(df_languages_devs,trim_pct =.1)
    
    # # Since we want to know what is a typical salary from each programming language, we'll filter our data removing
    # # those languages which had the median/mean ratio much less than 1, because this shows the salaries are in these cases
    # # vary a lot between the respondents
    df_languages_devs = filter_df_languages_devs(df_languages_devs,min_median_mean_ratio=.6,min_resp=100,n_head=5)
    df_languages_devs['year'] = df.name
    df_languages_devs.rename(columns={want_to_learn_col:'desired_language'},inplace=True)
    df_devs_all = pd.concat([df_devs_all,df_languages_devs])

    # Creating a dataframe with the languages students most want to learn
    df_answers_students = create_df_by_main_branch(df_answers_clean, 'I am a student who is learning to code')\
                    .dropna(subset=[want_to_learn_col])
    df_languages_students = create_wanted_languages_df(df_answers_students,
                                                        'students',
                                                        want_to_learn_col,
                                                        worked_with_col,
                                                        salary_col)
    df_languages_students = df_languages_students.sort_values('respondents',ascending=False).head(5)
    df_languages_students['year'] = df.name
    df_languages_students.rename(columns={want_to_learn_col:'desired_language'},inplace=True)
    df_students_all = pd.concat([df_students_all,df_languages_students])

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [5]:
# Plotting top 5 programming languages desired by students over years

fig = go.Figure()
fig.add_trace(go.Bar(x=df_students_all['year'],
                y=df_students_all['respondents'],
                name='Respondents',
                text=df_students_all['desired_language'],
                textposition='inside',
                textfont_color='ghostwhite',
                marker={
                    'color': df_students_all['respondents'], 
                    'colorscale': 'redor'}
                ))
fig.update_layout(
    title='Most Wanted Programming Languages by Students',
    xaxis_tickfont_size=16,
    yaxis=dict(
        title='Respondents',
        titlefont_size=16,
        tickfont_size=14,
    ),
)
fig.show()

In [6]:
# Plotting top 5 programming languages desired by well compensated programmers over years
 
fig = go.Figure()
fig.add_trace(go.Bar(x=df_devs_all['year'],
                y=df_devs_all['respondents'],
                name='Respondents',
                text=df_devs_all['desired_language'],
                textposition='inside',
                textfont_color='ghostwhite',
                marker={
                    'color': df_devs_all['respondents'], 
                    'colorscale': 'burg'}
                ))
fig.update_layout(
    title='Most Wanted Programming Languages by Well Compensated Devs',
    xaxis_tickfont_size=16,
    yaxis=dict(
        title='Respondents',
        titlefont_size=16,
        tickfont_size=14,
    ),
)
fig.show()

# 2. Do programmers who are compensated in USD earn higher salaries than those who earn in BRL?

In [7]:
def create_dev_types_df(df,salary_col):
    '''
    INPUT 
        df - a dataframe with answers to the StackOverflow survey
        salary_col - the name of the column with respondents salaries in df
    OUTPUT
        df_dev_types - a dataframe with how many respondents are from each dev type and their salaries
    '''
    dev_types = {}

    for i, row in df.iterrows():
        types = row['DevType'].split(';')
        for t in types:
            if t not in dev_types:
                dev_types[t] = {'respondents':1, 'salaries':[row[salary_col]]}
                continue
            elif t in dev_types:
                dev_types[t]['respondents'] += 1
                dev_types[t]['salaries'].append(row[salary_col])

    df_dev_types = pd.DataFrame.from_dict(dev_types, orient='index').reset_index()\
                            .rename(columns={'index': 'dev_type', 0: 'count'})
    return df_dev_types

def create_df_by_country_currency(df,country,currency_col,currency):
    '''
    INPUT 
        df - a dataframe with answers to the StackOverflow survey
        country - the country you want to create a new dataframe only with respondents from it
        currency_col - the name of the column with the country's currency
        currency - the name of the currency itself
    OUTPUT
        df - a new dataframe filtered based on `country`, `currency_col` and `currency`
    '''
    return df[(df['Country'] == country) & (df[currency_col] == currency)]

In [8]:
# Diffent country/salary_currency combinations we want to explore in the 2021 survey results
countries_currencies = [
    ['Brazil','BRL\tBrazilian real'],
    ['Brazil','USD\tUnited States dollar'],
    ['United States of America','USD\tUnited States dollar'],
]

df_devs_salaries = pd.DataFrame()
for i, cc in enumerate(countries_currencies):
    country, currency_col = cc[0], cc[1]

    # Creating initial dataframe based on country and currency
    df = create_df_by_country_currency(df_answers,country,'Currency',currency_col)

    # Cleaning data
    df.dropna(subset=['DevType','ConvertedCompYearly'],inplace=True)
    df = drop_outliers(df.reset_index(drop=True),'ConvertedCompYearly')

    # Creating a dataframe with salaries statistics
    df = create_dev_types_df(df,'ConvertedCompYearly')
    df = calculate_salaries_statistics(df,trim_pct =.1)
    df.drop(['respondents','respondents_for_calc','mean_salary','median_pct_of_mean'],axis=1,inplace=True)

    # Merging dataframes from different years
    if i == 0:
        df_devs_salaries = df
    else:
        df_devs_salaries = pd.merge(df_devs_salaries,df,on='dev_type',how='left')

df_devs_salaries.rename(columns={
    'median_salary_x':'median_salary_br_brl',
    'median_salary_y':'median_salary_br_usd',
    'median_salary':'median_salary_us_usd'
},inplace=True)

# Removing `Others` from dev types to mantain only explicit professional types
df_devs_salaries = df_devs_salaries.loc[df_devs_salaries['dev_type'] != 'Other (please specify):',:]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
# Plotting salaries in Brazil and in the US for different dev types

df_devs_salaries.sort_values('median_salary_br_brl',ascending=False,inplace=True)

fig = go.Figure()
fig.add_trace(go.Bar(x=df_devs_salaries['dev_type'].head(10),
                y=df_devs_salaries['median_salary_br_brl'].head(10),
                name='Brazil',
                marker_color='rgb(234, 129, 113)' #4
                ))
fig.add_trace(go.Bar(x=df_devs_salaries['dev_type'].head(10),
                y=df_devs_salaries['median_salary_us_usd'].head(10),
                name='US',
                marker_color='rgb(202, 82, 104)' #6
                ))
fig.update_layout(
    height=700,
    title={
        'text':'Salaries By Dev Type',
        'font':{'size':20}

        },
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='Salary (USD)',
        titlefont_size=16,
        tickfont_size=12,
    ),
    legend=dict(
        x=.9,
        y=1.015,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)',
        borderwidth=10
    ),
    barmode='group',
    bargap=0.15, 
    bargroupgap=0.1,
    font={'size':13}
)
fig.show()

# 2.1 Are Brazilians' salaries from those who earn in USD higher than those who are compensated in BRL?

In [10]:
# Plotting salaries for Brazilian professionals who are compensated in BRL and in USD

df_devs_salaries_brazil = df_devs_salaries[['dev_type','median_salary_br_brl','median_salary_br_usd']].dropna()

fig = go.Figure()
fig.add_trace(go.Bar(x=df_devs_salaries_brazil['dev_type'],
                y=df_devs_salaries_brazil['median_salary_br_brl'],
                name='Brazil-BRL',
                marker_color='rgb(242, 185, 196)'
                ))
fig.add_trace(go.Bar(x=df_devs_salaries_brazil['dev_type'],
                y=df_devs_salaries_brazil['median_salary_br_usd'],
                name='Brazil-USD',
                marker_color='rgb(229, 151, 185)'
                )) #2 e #4
fig.update_layout(
    height=700,
    title={
        'text':'Salaries By Dev Type (Brazil)',
        'font':{'size':20}

        },
    xaxis_tickfont_size=12,
    yaxis=dict(
        title='Salary (USD)',
        titlefont_size=16,
        tickfont_size=12,
    ),
    legend=dict(
        x=.88,
        y=1.015,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)',
        borderwidth=10
    ),
    barmode='group',
    bargap=0.15, 
    bargroupgap=0.1,
    font={'size':13}
)
fig.show()

# 3. Are salaries growing faster in Brazil or in the US?

In [11]:
def alter_fields_cols(df):
    '''
    INPUT 
        df - a dataframe with answers to the StackOverflow survey
    OUTPUT
        df - a new dataframe with necessary column's and field's names changed to proper concat
    '''
    df.rename(columns={salary_col:'compensation_usd',currency_col:'currency'},inplace=True)
    df['Country'].replace('United States of America','United States',inplace=True)
    df['currency'].replace('USD\tUnited States dollar','USD',inplace=True)
    df['currency'].replace('BRL\tBrazilian real','BRL',inplace=True)
    return df

In [12]:
df_all_years = pd.DataFrame()

for df in dfs:
    df['year'] = df.name
    salary_col, currency_col = get_cols_names(df)[2], get_cols_names(df)[3]

    # Removing some unimportant columns
    df = df[['year','Country',currency_col,salary_col]]

    # Droping rows with top 15% salaries (maybe they were typed incorrectly or are much higher than typical salaries)
    df_answers_clean = drop_outliers(df.reset_index(drop=True),salary_col).dropna(subset=[salary_col])

    # Different values in fields and columns in different datasets represent the same thing, so we must choose
    # standard values for they in order to properly concat and filter
    df_answers_clean = alter_fields_cols(df_answers_clean)
    df_all_years = pd.concat([df_all_years,df_answers_clean])

# Creating dataframes based on country and currency to plot median salaries over years
# (filling NaN with 0 because the first year is the baseline, which should be 0 instead of NaN)
df_all_years_br = create_df_by_country_currency(df_all_years,'Brazil','currency','BRL')\
                    .groupby('year').median('compensation_usd').reset_index()
df_all_years_br['pct_change'] = df_all_years_br['compensation_usd'].pct_change().fillna(0)*100
df_all_years_us = create_df_by_country_currency(df_all_years,'United States','currency','USD')\
                    .groupby('year').median('compensation_usd').reset_index()
df_all_years_us['pct_change'] = df_all_years_us['compensation_usd'].pct_change().fillna(0)*100

In [13]:
# Plotting % differences in median salaries in Brazil and in the US over years

fig = go.Figure()

fig.add_trace(go.Scatter(x=df_all_years_br['year'],
                y=df_all_years_br['pct_change'],
                name='Brazil-BRL',
                marker_color='rgb(245, 183, 142)',
                connectgaps=False
                ))
fig.add_trace(go.Scatter(x=df_all_years_us['year'],
                y=df_all_years_us['pct_change'],
                name='United States-USD',
                marker_color='rgb(221, 104, 108)',
                connectgaps=False
                )) #2 e #4
fig.update_layout(
    height=700,
    title={
        'text':'Salaries Growth Over Years',
        'font':{'size':20}
        },
    xaxis_tickfont_size=12,
    xaxis = dict(
        tickmode = 'array',
        tickvals = ['2019', '2020','2021'],
        ticktext = ['2019', '2020','2021']
    ),
    yaxis=dict(
        title='% Change',
        titlefont_size=16,
        tickfont_size=12,
    ),
    legend=dict(
        x=0,
        y=1.015,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)',
        borderwidth=10
    ),
    barmode='group',
    bargap=0.15, 
    bargroupgap=0.1,
    font={'size':13}
)
fig.show()